# Imports 

In [33]:
import pandas as pd
import numpy as np
from psaw import PushshiftAPI

from tqdm import tqdm
tqdm.pandas()

import datetime

from ast import literal_eval

from typing import List, Tuple, Union, Optional

# Settings and Configuration 

In [2]:
SRC_DATA   = '../../data/webpage_data/full_flipside_data_clean.csv'
COLS_FP    = '../../data/reddit_data/submission_columns.txt' # List of columns each reddit post should have
SUBREDDITS = ['politics', 'usnews', 'worldnews']

# Article Data Loading 

Load the scraped data, clean list of linked articles, and create list of queries

In [145]:
art_data = pd.read_csv(SRC_DATA)

In [19]:
art_data['linked_arts_clean'] = art_data['linked_arts_clean'].str.replace('\n', ',')
art_data['linked_arts_clean'] = art_data['linked_arts_clean'].apply(literal_eval)

In [22]:
# Create list of queries in the form (title, article url list, date) for scraping
queries = art_data[['title', 'linked_arts_clean', 'date']]
queries = [[query[0], subquery, query[2]] for query in queries.values for subquery in query[1]]

In [23]:
queries

[['Checks In The Mail',
  'https://www.politico.com/news/magazine/2020/03/19/trump-coronavirus-bailout-economy-plan-137680',
  'March 20, 2020'],
 ['Checks In The Mail',
  'https://www.foxnews.com/opinion/coronavirus-stimulus-package-congress-adam-brandon?utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A%20foxnews%2Fopinion%20%28Internal%20-%20Opinion%20-%20Mixed%29',
  'March 20, 2020'],
 ['Checks In The Mail',
  'https://theweek.com/articles/903076/tea-party-case-trump-stimulus',
  'March 20, 2020'],
 ['Checks In The Mail',
  'https://www.bloomberg.com/opinion/articles/2020-03-17/coronavirus-crisis-trump-s-1-2-trillion-stimulus-isn-t-enough',
  'March 20, 2020'],
 ['Checks In The Mail',
  'https://theweek.com/articles/903154/democrats-destructive-obsession-means-testing',
  'March 20, 2020'],
 ['EOs Regarding Transgender Rights',
  'https://www.spectator.co.uk/article/biden-s-trans-rights-agenda-is-bad-news-for-women',
  'January 26, 2021'],
 ['EOs Regarding Transgender Righ

# Reddit Scraping 

Scrape Reddit using the PushShift Reddit API Wrapper 

In [24]:
# Initialize api object
api = PushshiftAPI()

In [25]:
# Read pre-generated file holding the names of all columns a reddit post could contain
# Addresses issues with reddit posts that are missing data, allowing all data to be formatted correctly in a single file
with open(COLS_FP, 'r') as f:
    COLUMNS = literal_eval(f.read())

In [30]:
def get_posts_by_news(query: str, 
                      output_csv: str, 
                      title_or_url: str = 'query', 
                      subreddits: List[str] = ['politics'], 
                      date_thresh: int = 10, 
                      header: bool = True, 
                      max_posts: int = 10, 
                      retry: bool = True) -> None:
    '''
        Scrape reddit posts according to given query
        
        Parameters:
            -query
                Search query to fetch posts from reddit api
            -output_csv
                File path to store fetched posts
            -subreddits
                Specifies a list of subreddits to restrict result posts
            -date_thresh
                Specifies how long after the news article was published a post may be published
            -max_posts
                Maximum number of posts to scrape for a given query
            -retry
                If true, will retry query using only the first two words of the original query if no results are returned
                Primarily for queries with very specific titles that reddit has difficulty searching for
        
        Return:
            None, stores scraped data automatically
                
    '''
    
    title = query[0]
    url = query[1]
    date = datetime.datetime.strptime(query[2], '%B %d, %Y')
    
    if date_thresh:
        min_date = int((date - datetime.timedelta(days = date_thresh)).timestamp())
        max_date = int((date + datetime.timedelta(days = date_thresh)).timestamp())
    else:
        min_date = None
        max_date = None
    
    num_collected = 0
    for subreddit in subreddits:
#         print(url, min_date, max_date)
        if title_or_url == 'title':
            gen = api.search_submissions(subreddit = subreddit, q = title, after = min_date, before = max_date)
        elif title_or_url == 'url':
            gen = api.search_submissions(subreddit = subreddit, url = url, after = min_date, before = max_date)
        else:
            print('Must specify title or url')
            return
    
        for post in gen:
            data = post.d_
            data_df = pd.DataFrame({key:str(val) for key, val in data.items()}, index = [num_collected], columns = COLUMNS).fillna('None')
            data_df['title_q'] = title
            data_df['date_q'] = date

            data_df.to_csv(output_csv, mode = 'a', escapechar = '\\', header = header and num_collected == 0)
            num_collected += 1

            if num_collected > max_posts:
                break
    
    if num_collected == 0:
        print(f'Query {query} returned 0 results. {"Retrying with less terms" if retry and title_or_url == "title" else ""}')
        if retry and title_or_url == 'title':
            new_query = [' '.join(title.split(' ')[:2]), query[1], query[2]]
            get_posts_by_news(new_query, output_csv, title_or_url = title_or_url, subreddit = subreddit, date_thresh = date_thresh, header = header, max_posts = max_posts, retry = False)
    else:
        print(f'Query {query} returned {num_collected} results.')

In [31]:
# Run the scraping and store all results
OUTPUT_CSV = '../../data/reddit_data/reddit_scrape.csv'
WRITE_HEAD = True

# Iterate over queries, scrape, and save data
for i, query in enumerate(queries):
    get_posts_by_news(query, OUTPUT_CSV, title_or_url = 'url', subreddits = SUBREDDITS, date_thresh = 30, header = WRITE_HEAD, max_posts = 10, retry = True)
    print(f'Finished query {i} of {len(queries)}', end = '\r')
    WRITE_HEAD = False

Query ['Checks In The Mail', 'https://www.politico.com/news/magazine/2020/03/19/trump-coronavirus-bailout-economy-plan-137680', 'March 20, 2020'] returned 0 results. 
Query ['Checks In The Mail', 'https://www.foxnews.com/opinion/coronavirus-stimulus-package-congress-adam-brandon?utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A%20foxnews%2Fopinion%20%28Internal%20-%20Opinion%20-%20Mixed%29', 'March 20, 2020'] returned 0 results. 
Query ['Checks In The Mail', 'https://theweek.com/articles/903076/tea-party-case-trump-stimulus', 'March 20, 2020'] returned 0 results. 
Query ['Checks In The Mail', 'https://www.bloomberg.com/opinion/articles/2020-03-17/coronavirus-crisis-trump-s-1-2-trillion-stimulus-isn-t-enough', 'March 20, 2020'] returned 0 results. 
Query ['Checks In The Mail', 'https://theweek.com/articles/903154/democrats-destructive-obsession-means-testing', 'March 20, 2020'] returned 1 results.


C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 504
  warnings.warn("Got non 200 code %s" % response.status_code)


Query ['EOs Regarding Transgender Rights', 'https://www.spectator.co.uk/article/biden-s-trans-rights-agenda-is-bad-news-for-women', 'January 26, 2021'] returned 1 results.
Query ['EOs Regarding Transgender Rights', 'https://www.vox.com/identities/2021/1/25/22248553/trans-military-ban-revoked-biden', 'January 26, 2021'] returned 0 results. 
Query ['EOs Regarding Transgender Rights', 'https://apnews.com/article/biden-reverse-ban-transgender-military-f0ace4f9866e0ca0df021eba75b3af20', 'January 26, 2021'] returned 2 results.
Query ['EOs Regarding Transgender Rights', 'https://www.cnn.com/travel/article/cuisine-auntdai-restaurant-menu-comments-trnd/index.html', 'January 26, 2021'] returned 0 results. 
Query ['EOs Regarding Transgender Rights', 'https://apnews.com/article/montana-bills-judiciary-helena-060d274f606d9e8e08b786be0398efd3', 'January 26, 2021'] returned 0 results. 
Query ['Facebook and Breaking Up Big Tech', 'https://medium.com/@teamwarren/heres-how-we-can-break-up-big-tech-9ad9e

Query ['Taiwan', 'https://theweek.com/world/1005622/what-is-china-doing-over-taiwan', 'October 6, 2021'] returned 0 results. 
Query ['Taiwan', 'https://www.brightcellars.com/take-wine-quiz/?utm_campaign=NLETTSSthe-flip-side&utm_source=nlett&utm_medium=social&utm_content=6&?n_id=nlett&a_id=NLETTSSthe-flip-side&d=2&sixbtl=bc&bt=881', 'October 6, 2021'] returned 0 results. 
Query ['Taiwan', 'https://apnews.com/article/china-taiwan-43c581adbe22c82a2af51c7b009717cd', 'October 6, 2021'] returned 0 results. 
Query ['Taiwan', 'https://www.washingtonexaminer.com/opinion/op-eds/recognize-taiwan', 'October 6, 2021'] returned 0 results. 
Query ['Tech Sector Update', 'https://hotair.com/john-s-2/2020/12/09/ftc-46-states-file-lawsuit-accusing-facebook-maintaining-monopoly-n377479', 'December 10, 2020'] returned 0 results. 
Query ['Tech Sector Update', 'https://www.demilked.com/funny-kid-school-photo-fails/', 'December 10, 2020'] returned 0 results. 
Query ['Tech Sector Update', 'https://hotair.com/e

Query ['ACA Declared Unconstitutional', 'https://www.latimes.com/opinion/editorials/la-ed-aca-unconstitutional-20181215-story.html', 'December 17, 2018'] returned 0 results. 
Query ['ACA Declared Unconstitutional', 'https://www.wsj.com/articles/texas-obamacare-blunder-11544996418', 'December 17, 2018'] returned 1 results.
Query ['AG Nominee William Barr', 'https://www.theflipside.io/archives/special-edition-ask-us-anything', 'January 16, 2019'] returned 0 results. 
Query ['AG Nominee William Barr', 'https://hotair.com/archives/2019/01/15/no-holds-barr-ed-confirmation-hearings-begin-two-lindseys/', 'January 16, 2019'] returned 0 results. 
Query ['AG Nominee William Barr', 'https://www.bloomberg.com/opinion/articles/2019-01-15/barr-hearing-trump-and-mueller-should-both-note-his-independence?srnd=politics-vp', 'January 16, 2019'] returned 0 results. 
Query ['AG Nominee William Barr', 'https://www.washingtonpost.com/opinions/confirm-william-barr--and-hold-him-to-his-pledge-of-independence/

...

Query ['Faithless Electors', 'https://www.scotusblog.com/2020/07/opinion-analysis-court-upholds-faithless-elector-laws/', 'July 7, 2020'] returned 0 results. 
Query ['Faithless Electors', 'https://hotair.com/ed-morrissey/2020/07/06/unanimous-scotus-electoral-college-keep-faith-else-n345286', 'July 7, 2020'] returned 0 results. 
Query ['Faithless Electors', 'https://reason.com/volokh/2020/07/06/steady-as-she-goes-on-faithless-electors/', 'July 7, 2020'] returned 0 results. 
Query ['Faithless Electors', 'https://www.boredpanda.com/self-sustaining-ecosystems-jardimnopote/', 'July 7, 2020'] returned 0 results. 
Query ['Withdrawal from Afghanistan', 'https://people.com/pets/rescue-dog-helps-raise-orphaned-kittens/', 'July 7, 2021'] returned 0 results. 
Query ['Withdrawal from Afghanistan', 'https://www.voanews.com/south-central-asia/us-troop-withdrawal-afghanistan-90-complete', 'July 7, 2021'] returned 0 results. 
Query ['Withdrawal from Afghanistan', 'https://slate.com/news-and-politics/20

Query ['Bill Russell and Nichelle Nichols', 'https://www.cnn.com/2022/08/01/opinions/bill-russell-basketball-nba-boston-racism-joseph/index.html', 'August 3, 2022'] returned 0 results. 
Query ['Bill Russell and Nichelle Nichols', 'https://www.newyorker.com/culture/postscript/bill-russell-was-basketballs-adam', 'August 3, 2022'] returned 0 results. 
Query ['Bill Russell and Nichelle Nichols', 'https://www.espn.com/olympics/story/_/id/34245374/ioc-reinstates-jim-thorpe-sole-winner-1912-olympic-decathlon-pentathlon', 'August 3, 2022'] returned 0 results. 
Query ['Bill Russell and Nichelle Nichols', 'https://redstate.com/jimthompson/2022/07/31/in-memoriam-bill-russell-n605059', 'August 3, 2022'] returned 0 results. 
Query ['Beirut', 'https://www.washingtonexaminer.com/opinion/beirut-explosion-dont-let-lebanons-political-order-shirk-accountability', 'August 11, 2020'] returned 0 results. 
Query ['Beirut', 'https://www.washingtonpost.com/opinions/2020/08/05/beirut-mourns-our-failed-leadershi

Query ['Middle East policy', 'https://www.reuters.com/article/uk-iraq-security-idUSKBN2AF1SP', 'February 19, 2021'] returned 0 results. 
Query ['Middle East policy', 'https://www.theflipside.io/archives/iran-enriches-uranium', 'February 19, 2021'] returned 0 results. 
Query ['Wisconsin and Michigan', 'https://thefederalist.com/2020/09/14/in-wisconsin-campaign-stop-mike-pence-says-2020-is-about-whether-america-remains-america/', 'September 17, 2020'] returned 0 results. 
Query ['Wisconsin and Michigan', 'https://www.realclearpolitics.com/epolls/2020/president/mi/michigan_trump_vs_biden-6761.html', 'September 17, 2020'] returned 0 results. 
Query ['Wisconsin and Michigan', 'https://nbcmontana.com/news/offbeat/mountain-community-celebrates-bigfoot-as-social-distancing-champ', 'September 17, 2020'] returned 0 results. 
Query ['Wisconsin and Michigan', 'https://www.thenation.com/article/economy/kenosha-election/', 'September 17, 2020'] returned 1 results.
Query ['Wisconsin and Michigan', 'h

Query ['Federal Reserve and Jobs Report', 'https://www.wsj.com/articles/jerome-powell-puts-paul-volcker-on-hold-federal-reserve-interest-rates-inflation-11651702723', 'May 9, 2022'] returned 0 results. 
Query ['Federal Reserve and Jobs Report', 'https://apnews.com/article/russia-ukraine-covid-health-business-europe-394731ffe3540ad140de880127879de5', 'May 9, 2022'] returned 0 results. 
Query ['Federal Reserve and Jobs Report', 'https://www.washingtonpost.com/opinions/2022/05/06/stock-market-decline-not-all-bad-news', 'May 9, 2022'] returned 0 results. 
Query ['Federal Reserve and Jobs Report', 'https://turnto10.com/news/instagram/tuskegee-airman-asks-for-birthday-cards-ahead-of-100-birthday-next-month-rhode-island-world-war-american-airforce-veteran', 'May 9, 2022'] returned 0 results. 
Query ['Federal Reserve and Jobs Report', 'https://prospect.org/blogs-and-newsletters/tap/feds-dilemma-interest-rate-hike/', 'May 9, 2022'] returned 0 results. 
Query ['Shooting in Buffalo', 'https://www

Query ['Biden Visits Saudi Arabia', 'https://spectatorworld.com/topic/what-is-biden-returning-to-the-middle-east/', 'July 15, 2022'] returned 0 results. 
Query ['Biden Visits Saudi Arabia', 'https://www.boredpanda.com/stray-dog-comes-to-the-wedding-gets-adopted-caramelodejesus', 'July 15, 2022'] returned 0 results. 
Query ['Biden Visits Saudi Arabia', 'https://www.cato.org/commentary/president-joe-biden-heads-mideast-plans-abase-himself-saudi-royals', 'July 15, 2022'] returned 0 results. 


In [38]:
reddit_posts = pd.read_csv(OUTPUT_CSV, header = None)
reddit_posts = reddit_posts.drop(0, axis = 1)

In [46]:
reddit_posts.columns = list(COLUMNS) + ['title_q', 'date_q']

In [47]:
reddit_posts.groupby('title_q').count().sort_values('author', ascending = True)

,all_awardings,allow_live_comments,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,title,total_awards_received,treatment_tags,upvote_ratio,url,url_overridden_by_dest,whitelist_status,wls,created,date_q
title_q,,,,,,,,,,,,,,,,,,,,,
YouTube's Crowder Controversy,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Debate Night I,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Debate Night II,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Oil Price War,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Obamacare in Jeopardy,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Joe Biden,17,17,17,10,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17
Election Update,19,19,19,16,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
Final Pre-Election Update,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,20,20


In [48]:
reddit_short = reddit_posts[['title_q', 'date_q', 'title', 'selftext', 'upvote_ratio']]

## Rescrape Missing Queries 

Retry reddit scrape with more relaxed date thresholds to fill in gaps of queries with no results 

In [73]:
title_counts = reddit_short.groupby('title_q').count()
titles = title_counts[title_counts['title'] > 10].index

In [74]:
src_titles = art_data['title'].str.strip().unique()

In [75]:
miss_titles = set(src_titles).difference(titles)
miss_queries = [list(query) for query in queries if query[0] in miss_titles]

In [77]:
OUTPUT_CSV = '../../data/reddit_data/reddit_scrape.csv'
for i, query in enumerate(miss_queries):
    get_posts_by_news(query, OUTPUT_CSV, title_or_url = 'url', date_thresh = 30, subreddits = SUBREDDITS, header = False, max_posts = 10)
    print(f'Finished query {i} of {len(miss_queries)}', end = '\r')

C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


Query ['Checks In The Mail', 'https://www.politico.com/news/magazine/2020/03/19/trump-coronavirus-bailout-economy-plan-137680', 'March 20, 2020'] returned 0 results. 
Query ['Checks In The Mail', 'https://www.foxnews.com/opinion/coronavirus-stimulus-package-congress-adam-brandon?utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A%20foxnews%2Fopinion%20%28Internal%20-%20Opinion%20-%20Mixed%29', 'March 20, 2020'] returned 0 results. 


C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


Query ['Checks In The Mail', 'https://theweek.com/articles/903076/tea-party-case-trump-stimulus', 'March 20, 2020'] returned 0 results. 
Query ['Checks In The Mail', 'https://www.bloomberg.com/opinion/articles/2020-03-17/coronavirus-crisis-trump-s-1-2-trillion-stimulus-isn-t-enough', 'March 20, 2020'] returned 0 results. 
Query ['Checks In The Mail', 'https://theweek.com/articles/903154/democrats-destructive-obsession-means-testing', 'March 20, 2020'] returned 1 results.
Query ['EOs Regarding Transgender Rights', 'https://www.spectator.co.uk/article/biden-s-trans-rights-agenda-is-bad-news-for-women', 'January 26, 2021'] returned 1 results.
Query ['EOs Regarding Transgender Rights', 'https://www.vox.com/identities/2021/1/25/22248553/trans-military-ban-revoked-biden', 'January 26, 2021'] returned 0 results. 
Query ['EOs Regarding Transgender Rights', 'https://apnews.com/article/biden-reverse-ban-transgender-military-f0ace4f9866e0ca0df021eba75b3af20', 'January 26, 2021'] returned 2 result

Query ['Tech Sector Update', 'https://apnews.com/article/youtube-election-misinformation-removal-74ca3738e2774c9a4cf8fbd1e977710f', 'December 10, 2020'] returned 0 results. 
Query ['Vaccine Booster Shots', 'https://apnews.com/article/coronavirus-vaccine-booster-shots-a10daad08f6d47aa5beca0b4da9da9d2', 'August 23, 2021'] returned 0 results. 
Query ['Vaccine Booster Shots', 'https://slate.com/technology/2021/08/booster-vaccine-coronavirus-individualism-fda.html', 'August 23, 2021'] returned 0 results. 
Query ['Vaccine Booster Shots', 'https://www.washingtonpost.com/opinions/2021/08/18/why-biden-administrations-recommendation-booster-shots-is-mistake/', 'August 23, 2021'] returned 0 results. 
Query ['Vaccine Booster Shots', 'https://www.washingtonexaminer.com/opinion/surgeon-general-murthy-is-right-to-be-a-booster-for-booster-shots', 'August 23, 2021'] returned 0 results. 
Query ['Vaccine Booster Shots', 'https://www.cdc.gov/coronavirus/2019-ncov/vaccines/booster-shot.html', 'August 23, 2

...

Query ['Darya Dugina', 'https://www.washingtonexaminer.com/opinion/russias-internal-shadow-war-intensifies', 'August 24, 2022'] returned 0 results. 
Query ['Darya Dugina', 'https://thenextweb.com/news/janet-jackson-rhythm-nation-causes-laptops-to-crash-vibration-resonance', 'August 24, 2022'] returned 0 results. 
Query ['Darya Dugina', 'https://www.cnn.com/2022/08/23/opinions/darya-dugina-alexander-dugin-putin-rutland/index.html', 'August 24, 2022'] returned 0 results. 
Query ['Darya Dugina', 'https://thedispatch.com/p/questions-surround-the-death-of-daria', 'August 24, 2022'] returned 0 results. 
Query ['Darya Dugina', 'https://www.reuters.com/world/europe/russias-fsb-says-ukraine-behind-murder-ultra-nationalists-daughter-2022-08-22/', 'August 24, 2022'] returned 2 results.
Query ['Chaos in Afghanistan', 'https://www.politico.com/news/magazine/2021/08/16/afghan-leadership-us-save-country-505160', 'August 17, 2021'] returned 0 results. 
Query ['Chaos in Afghanistan', 'https://apnews.co

Query ['Violence at the Capitol', 'https://www.foxnews.com/opinion/capitol-attack-trump-supporters-deroy-murdock', 'January 7, 2021'] returned 0 results. 
Query ['Violence at the Capitol', 'https://www.thenation.com/article/politics/capitol-protest-trump/', 'January 7, 2021'] returned 1 results.
Query ['Violence at the Capitol', 'https://twitter.com/RepAndyBiggsAZ/status/1346512586741854219', 'January 7, 2021'] returned 0 results. 
Query ['Violence at the Capitol', 'https://www.foxnews.com/opinion/tucker-carlson-capitol-hill-riots-reaction', 'January 7, 2021'] returned 1 results.
Query ['Violence at the Capitol', 'https://www.theamericanconservative.com/dreher/religious-meaning-of-trump-maga-riot/', 'January 7, 2021'] returned 0 results. 
Query ['Oscars Diversity Requirements', 'https://www.reuters.com/article/awards-oscars-diversity/oscars-academy-sets-out-new-diversity-standards-for-best-picture-contenders-idUSL1N2G600Q', 'September 11, 2020'] returned 0 results. 
Query ['Oscars Dive

Query ['China and Russia', 'https://hotair.com/allahpundit/2022/03/14/chinese-think-tank-china-should-cut-putin-loose-asap-and-make-nice-with-the-invigorated-west-n455192', 'March 15, 2022'] returned 0 results. 
Query ['Afghanistan Troop Withdrawal', 'https://www.goodnewsnetwork.org/mit-scientists-help-spiders-make-music/', 'April 15, 2021'] returned 0 results. 
Query ['Afghanistan Troop Withdrawal', 'https://www.foxnews.com/opinion/afghanistan-20-years-war-daniel-davis', 'April 15, 2021'] returned 0 results. 
Query ['Afghanistan Troop Withdrawal', 'https://www.foxnews.com/opinion/biden-afghanistan-politics-adam-kinzinger', 'April 15, 2021'] returned 0 results. 
Query ['Afghanistan Troop Withdrawal', 'https://nymag.com/intelligencer/2021/04/biden-all-u-s-troops-will-leave-afghanistan-by-9-11.html', 'April 15, 2021'] returned 0 results. 
Query ['Afghanistan Troop Withdrawal', 'https://www.theflipside.io/archives/afghanistan-icc', 'April 15, 2021'] returned 0 results. 
Query ['Amash Cons

Query ['Pelosi Goes To Taiwan', 'https://www.reuters.com/world/asia-pacific/pelosi-addresses-taiwan-parliament-visit-condemned-by-china-2022-08-03/', 'August 5, 2022'] returned 3 results.
Query ['Pelosi Goes To Taiwan', 'https://www.nationalreview.com/2022/08/pelosis-courageous-trip-to-taiwan/', 'August 5, 2022'] returned 0 results. 
Query ['Johnny Depp and Amber Heard', 'https://www.washingtonexaminer.com/opinion/johnny-depps-testimony-sheds-light-on-male-victims-of-domestic-abuse', 'May 6, 2022'] returned 0 results. 
Query ['Johnny Depp and Amber Heard', 'https://www.nationalreview.com/2022/04/johnny-depps-spectacular-self-demolition/', 'May 6, 2022'] returned 0 results. 
Query ['Johnny Depp and Amber Heard', 'https://www.washingtonexaminer.com/opinion/shocking-moments-from-the-johnny-depp-libel-trial', 'May 6, 2022'] returned 0 results. 
Query ['Johnny Depp and Amber Heard', 'https://www.vox.com/culture/23043519/johnny-depp-amber-heard-defamation-trial-fairfax-county-domestic-abuse-

Query ['Apple to Scan for CSAM', 'https://people.com/home/ikea-swedish-meatball-scented-candles-gift-box-for-loyalty-program/', 'August 9, 2021'] returned 0 results. 
Query ['It was the best of times, it was the worst of times', 'https://www.usatoday.com/story/opinion/2020/10/20/why-americans-more-united-even-election-year-than-appears-column/3649525001/', 'November 5, 2020'] returned 0 results. 
Query ['It was the best of times, it was the worst of times', 'https://www.microsoft.com/en-us/p/la-noire/bqdvbcs5z4n1', 'November 5, 2020'] returned 0 results. 
Query ['It was the best of times, it was the worst of times', 'https://store.steampowered.com/app/110800/LA_Noire/', 'November 5, 2020'] returned 0 results. 
Query ['It was the best of times, it was the worst of times', 'https://bananagrams.com/products/bananagrams', 'November 5, 2020'] returned 0 results. 
Query ['It was the best of times, it was the worst of times', 'https://ballotpedia.org/Ballotpedia%27s_2020_Election_Help_Desk', 

In [88]:
reddit_posts2 = pd.read_csv(OUTPUT_CSV, header = None)
reddit_posts2 = reddit_posts2.drop(0, axis = 1)
reddit_posts2.columns = COLUMNS + ['title_q', 'date_q']

In [89]:
reddit_posts2

,all_awardings,allow_live_comments,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,total_awards_received,treatment_tags,upvote_ratio,url,url_overridden_by_dest,whitelist_status,wls,created,title_q,date_q
0,[],False,[deleted],NaN,None,None,None,None,dark,None,...,0,None,None,https://theweek.com/articles/903154/democrats-...,None,all_ads,6,1.584671e+09,Checks In The Mail,2020-03-20
1,[],False,linseedhobbit,None,None,[],None,None,None,text,...,0,[],1.0,https://www.spectator.co.uk/article/biden-s-tr...,https://www.spectator.co.uk/article/biden-s-tr...,all_ads,6,1.612211e+09,EOs Regarding Transgender Rights,2021-01-26
2,[],False,[deleted],NaN,None,None,None,None,dark,None,...,0,None,None,https://theweek.com/articles/903154/democrats-...,None,all_ads,6,1.584671e+09,Checks In The Mail,2020-03-20
3,[],False,linseedhobbit,None,None,[],None,None,None,text,...,0,[],1.0,https://www.spectator.co.uk/article/biden-s-tr...,https://www.spectator.co.uk/article/biden-s-tr...,all_ads,6,1.612211e+09,EOs Regarding Transgender Rights,2021-01-26
4,[],False,[deleted],NaN,None,None,None,None,dark,None,...,0,None,None,https://theweek.com/articles/903154/democrats-...,None,all_ads,6,1.584671e+09,Checks In The Mail,2020-03-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3043,[],False,BobbingForSmegma,None,None,[],None,None,None,text,...,0,[],0.94,https://www.vox.com/2021/6/17/22538462/supreme...,https://www.vox.com/2021/6/17/22538462/supreme...,all_ads,6,1.623964e+09,Supreme Court Rulings,2021-06-18
3044,[],False,Crafty_Programmer,None,None,[],None,None,None,text,...,0,[],1.0,https://www.eff.org/deeplinks/2021/08/apples-p...,https://www.eff.org/deeplinks/2021/08/apples-p...,all_ads,6,1.628216e+09,Apple to Scan for CSAM,2021-08-09
3045,[],False,Cosimo_68,None,None,[],None,None,None,text,...,0,[],1.0,https://www.eff.org/deeplinks/2021/08/apples-p...,https://www.eff.org/deeplinks/2021/08/apples-p...,all_ads,6,1.628311e+09,Apple to Scan for CSAM,2021-08-09
3046,[],False,AnotherScrubName,None,None,[],None,None,None,text,...,0,[],1.0,https://www.cnn.com/2021/08/18/politics/americ...,https://www.cnn.com/2021/08/18/politics/americ...,all_ads,6,1.629358e+09,Afghanistan and the World,2021-08-19


In [90]:
reddit_short2 = reddit_posts2[['title_q', 'date_q', 'title', 'selftext', 'upvote_ratio']]

# Scraping by Title

In [146]:
all_titles = [[row[0], '', row[1]] for row in art_data[['title', 'date']].values]

In [151]:
title_queries = [query for query in all_titles if query[0]]

In [152]:
# Run the scraping and store all results
OUTPUT_CSV = '../../data/reddit_data/title_reddit_scrape.csv'
WRITE_HEAD = True
for i, query in enumerate(title_queries):
    get_posts_by_news(query, OUTPUT_CSV, title_or_url = 'url', subreddits = SUBREDDITS, date_thresh = 10, header = WRITE_HEAD, max_posts = 10, retry = True)
    print(f'Finished query {i} of {len(title_queries)}', end = '\r')
    WRITE_HEAD = False

C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


Query ['Checks In The Mail', '', 'March 20, 2020'] returned 13 results.
Query ['EOs Regarding Transgender Rights', '', 'January 26, 2021'] returned 13 results.
Query ['Facebook and Breaking Up Big Tech', '', 'April 2, 2019'] returned 13 results.


C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


Query ['General Election Update', '', 'October 28, 2020'] returned 13 results.
Query ['Impeachment Hearings Continue', '', 'November 22, 2019'] returned 13 results.
Query ['Inflation', '', 'January 25, 2022'] returned 13 results.
Query ['Judy Shelton', '', 'November 18, 2020'] returned 13 results.
Query ['Latest Polling and Dems 2020 Update', '', 'June 13, 2019'] returned 13 results.
Query ['Meng Wanzhou Arrested', '', 'December 7, 2018'] returned 13 results.
Query ['NYT Scoop and NYS Bill re Trump’s Taxes', '', 'May 9, 2019'] returned 13 results.
Query ['Taiwan', '', 'October 6, 2021'] returned 13 results.
Query ['Tech Sector Update', '', 'December 10, 2020'] returned 13 results.
Query ['Vaccine Booster Shots', '', 'August 23, 2021'] returned 13 results.
Query ['Woodward’s Trump Interviews', '', 'September 10, 2020'] returned 13 results.
Query ['1st 2020 Democratic Debate', '', 'June 27, 2019'] returned 13 results.
Query ['2020 Census Battle', '', 'June 5, 2019'] returned 13 results.


Query ['Breonna Taylor', '', 'September 24, 2020'] returned 13 results.
Query ['Brexit', '', 'January 31, 2020'] returned 13 results.
Query ['Brexit Countdown', '', 'April 8, 2019'] returned 13 results.
Query ['Brexit Delayed Until After British Vote', '', 'October 30, 2019'] returned 13 results.
Query ['Brittney Griner', '', 'July 12, 2022'] returned 13 results.
Query ['Budget Deal Reached', '', 'July 24, 2019'] returned 13 results.
Query ['Build Back Better Stalls', '', 'December 20, 2021'] returned 13 results.
Query ['Bullet Train', '', 'February 14, 2019'] returned 13 results.
Query ['Buttigieg Criticizes Pence', '', 'April 10, 2019'] returned 13 results.
Query ['CDC Shakeup', '', 'August 19, 2022'] returned 13 results.
Query ['CNN Files Lawsuit Against WH', '', 'November 16, 2018'] returned 13 results.
Query ['CNN Town Hall', '', 'September 6, 2019'] returned 13 results.
Query ['COVID Relief Bill', '', 'December 3, 2020'] returned 13 results.
Query ['COVID Stimulus', '', 'February

Query ['Facebook Censorship', '', 'May 6, 2019'] returned 13 results.
Query ['Facebook in Australia', '', 'February 25, 2021'] returned 13 results.
Query ['Facebook’s Political Ad Policy', '', 'October 31, 2019'] returned 13 results.
Query ['Fauci’s Emails', '', 'June 10, 2021'] returned 13 results.
Query ['Federal Reserve Acts', '', 'June 17, 2022'] returned 13 results.
Query ['Federal Vaccine Mandate', '', 'September 10, 2021'] returned 13 results.
Query ['Final Debate', '', 'October 23, 2020'] returned 13 results.
Query ['Final Pre-Election Update', '', 'November 2, 2020'] returned 13 results.
Query ['Florida Education Bill', '', 'March 10, 2022'] returned 13 results.
Query ['Florida Recounts', '', 'November 14, 2018'] returned 13 results.
Query ['Florida and Disney', '', 'April 7, 2022'] returned 13 results.
Query ['Food Stamps', '', 'December 6, 2019'] returned 13 results.
Query ['Ford and Kavanaugh Testify', '', 'September 28, 2018'] returned 13 results.
Query ['Foreign Policy Ch

Query ['Pete Buttigieg Rising', '', 'November 15, 2019'] returned 13 results.
Query ['Pipe Bombs Sent to High-Profile Democrats', '', 'October 25, 2018'] returned 13 results.
Query ['Pittsburgh Synagogue Massacre', '', 'October 29, 2018'] returned 13 results.
Query ['Planned Parenthood Exits Title X Program', '', 'August 21, 2019'] returned 13 results.
Query ['Police Protests', '', 'June 1, 2020'] returned 13 results.
Query ['Portland', '', 'July 21, 2020'] returned 13 results.
Query ['Possible Tariffs on Mexican Goods', '', 'June 3, 2019'] returned 13 results.
Query ['Presidential Debate Chaos', '', 'October 9, 2020'] returned 13 results.
Query ['Presidential debate', '', 'September 30, 2020'] returned 13 results.
Query ['Primary Elections', '', 'August 11, 2022'] returned 13 results.
Query ['Proposals to Prevent Future Shootings', '', 'August 6, 2019'] returned 13 results.
Query ['RNC Censure', '', 'February 9, 2022'] returned 13 results.
Query ['Race and Coronavirus', '', 'April 13,

Query ['The Facebook Papers', '', 'October 27, 2021'] returned 13 results.
Query ['The Future of Brexit', '', 'January 17, 2019'] returned 13 results.
Query ['The Latino Vote', '', 'September 18, 2020'] returned 13 results.
Query ['The Lincoln Project', '', 'August 3, 2020'] returned 13 results.
Query ['Tim Scott', '', 'May 4, 2021'] returned 13 results.
Query ['Title 42', '', 'April 4, 2022'] returned 13 results.
Query ['Title IX', '', 'May 8, 2020'] returned 13 results.
Query ['Top Gun', '', 'June 1, 2022'] returned 13 results.
Query ['Trade Deficit Surges', '', 'March 11, 2019'] returned 13 results.
Query ['Trade War With China', '', 'May 16, 2019'] returned 13 results.
Query ['Troop Withdrawal From Germany', '', 'August 4, 2020'] returned 13 results.
Query ['Trump Acquitted by Senate', '', 'February 16, 2021'] returned 13 results.
Query ['Trump Announces Continued Support for Saudi Government', '', 'November 21, 2018'] returned 13 results.
Query ['Trump Banned From Twitter', '', 'J

Query ['Disinformation Board', '', 'May 3, 2022'] returned 13 results.
Query ['Whoopi Goldberg', '', 'February 3, 2022'] returned 13 results.
Query ['Michael Flynn', '', 'May 4, 2020'] returned 13 results.
Query ['Israel’s New Coalition', '', 'June 4, 2021'] returned 13 results.
Query ['Ilhan Omar Under Fire', '', 'March 5, 2019'] returned 13 results.
Query ['TikTok', '', 'August 5, 2020'] returned 13 results.
Query ['Liz Cheney', '', 'May 5, 2021'] returned 13 results.
Query ['Eviction Moratorium', '', 'August 5, 2021'] returned 13 results.
Query ['Hong Kong', '', 'June 5, 2020'] returned 13 results.
Query ['Independence Day', '', 'July 5, 2022'] returned 13 results.
Query ['Georgia Senate Runoffs', '', 'January 6, 2021'] returned 13 results.
Query ['Captain Crozier', '', 'April 6, 2020'] returned 13 results.
Query ['Biden’s Speech', '', 'September 6, 2022'] returned 13 results.
Query ['Chesa Boudin', '', 'June 6, 2022'] returned 13 results.
Query ['Inflation', '', 'May 13, 2021'] ret

Query ['Congressional Stock Trading', '', 'February 14, 2022'] returned 13 results.
Query ['Israel’s New Coalition', '', 'June 15, 2021'] returned 13 results.
Query ['Salman Rushdie', '', 'August 15, 2022'] returned 13 results.
Query ['Supreme Court Rules on LGBTQ Rights', '', 'June 16, 2020'] returned 13 results.
Query ['Afghanistan', '', 'August 16, 2022'] returned 13 results.
Query ['Fall of Kabul', '', 'August 16, 2021'] returned 13 results.
Query ['Russia and Ukraine', '', 'April 16, 2021'] returned 13 results.
Query ['White House Transition', '', 'November 16, 2020'] returned 13 results.
Query ['States Reopening', '', 'May 21, 2020'] returned 13 results.
Query ['Haitian Migrants', '', 'September 21, 2021'] returned 13 results.
Query ['Religious Schools in Maine', '', 'June 22, 2022'] returned 13 results.
Query ['SCOTUS Rules on Student Athletes', '', 'June 22, 2021'] returned 13 results.
Query ['Virginia Governor’s Race', '', 'October 22, 2021'] returned 13 results.
Query ['Overd

# Consolidation 

In [30]:
URL_DATA_FP   = '../../data/reddit_data/reddit_scrape.csv'
TITLE_DATA_FP = '../../data/reddit_data/title_reddit_scrape.csv'
FINAL_OUT_FP  = '../../data/reddit_data/all_reddit.csv'

In [17]:
url_data = pd.read_csv(URL_DATA_FP, header = None)
title_data = pd.read_csv(TITLE_DATA_FP)

In [18]:
title_data = title_data.drop('Unnamed: 0', axis = 1)
url_data   = url_data.drop(0, axis = 1)
url_data.columns = title_data.columns

In [21]:
all_reddit = pd.concat([title_data, url_data], axis = 0)

In [29]:
all_reddit = all_reddit.drop_duplicates(subset = 'id')

In [31]:
all_reddit.to_csv(FINAL_OUT_FP, index = None)